## 1 Conexión a la Nube
Esta celda carga tus credenciales secretas y prueba si Google te deja entrar.

In [2]:
import os
import pandas as pd
import glob
import json
from google.cloud import bigquery
from google.oauth2 import service_account

# --- CONFIGURACIÓN ---
KEY_PATH = "../gcp_credentials.json"  # La llave está una carpeta atrás
PROJECT_ID = "urban-lifestyle-data"   # Tu ID de proyecto
REGION = "EU" # O "US", depende de donde estés

print(f"🔑 Buscando llave en: {os.path.abspath(KEY_PATH)}")

if os.path.exists(KEY_PATH):
    print("✅ Archivo de credenciales encontrado.")
    
    # Conectar
    credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
    client = bigquery.Client(credentials=credentials, project=PROJECT_ID)
    print("✅ Cliente de BigQuery iniciado correctamente.")
else:
    print("❌ ERROR: No encuentro gcp_credentials.json. Verifica que esté en la raiz del proyecto.")

🔑 Buscando llave en: c:\Users\luisa\Documents\NORRDATA\proyectos\urban-lifestyle-data\gcp_credentials.json
✅ Archivo de credenciales encontrado.
✅ Cliente de BigQuery iniciado correctamente.


## 2: Auditoría de VENTAS (Volumen y Suciedad)

Vamos a ver cuántos archivos generamos y abrir uno al azar para ver qué tan "sucio" está.

In [3]:
# Buscar todos los CSVs de ventas
sales_files = glob.glob("../data/raw/sales/*.csv")
print(f"📂 Total de archivos de ventas encontrados: {len(sales_files)}")

# Cargar el PRIMERO para ver la estructura
if sales_files:
    sample_file = sales_files[0]
    df_sales = pd.read_csv(sample_file)
    
    print(f"\n--- 🕵️ INSPECCIÓN DE: {os.path.basename(sample_file)} ---")
    display(df_sales.head())
    
    print("\n--- ⚠️ REPORTE DE CALIDAD ---")
    print(df_sales.info())
    
    # Detectar problemas típicos
    nulls = df_sales.isnull().sum().sum()
    dirty_prices = df_sales['price'].apply(lambda x: isinstance(x, str)).sum()
    
    print(f"\n💣 PROBLEMAS DETECTADOS:")
    print(f"   - Valores Nulos totales: {nulls}")
    print(f"   - Precios como Texto (ej: '150 kr'): {dirty_prices} filas")
    print(f"   - Fechas con formato incorrecto (ej: '/'): {df_sales['ts'].str.contains('/').sum()} filas")

📂 Total de archivos de ventas encontrados: 1448

--- 🕵️ INSPECCIÓN DE: 2024-01-01_Online_HQ.csv ---


,tx_id,ts,store,prod,qty,price,pay_type
0,7b325aa7-6aef-4ea8-b39f-fe0d49c2fd82,2024-01-01 17:07:50,Online_HQ,ITEM-0094,3,34.43,Swish
1,3afbeb6b-242c-4a33-bee4-f5dd0d23b39a,2024-01-01 16:19:59,Online_HQ,ITEM-0121,3,119.16,Swish
2,ed87984e-b204-4dd3-9317-c23a59802e43,2024-01-01 02:46:04,Online_HQ,ITEM-0024,-1,175.2,NaN
3,9adb273a-1567-49e9-87d0-306b311aa974,2024-01-01 01:36:58,Online_HQ,ITEM-0073,4,101.4,Cash
4,e8ae65f1-f8d3-4c17-99e8-ce2ce69e914b,2024-01-01 21:51:41,Online_HQ,ITEM-0057,-1,134.98,Card



--- ⚠️ REPORTE DE CALIDAD ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tx_id     652 non-null    object
 1   ts        652 non-null    object
 2   store     652 non-null    object
 3   prod      652 non-null    object
 4   qty       652 non-null    int64 
 5   price     652 non-null    object
 6   pay_type  484 non-null    object
dtypes: int64(1), object(6)
memory usage: 35.8+ KB
None

💣 PROBLEMAS DETECTADOS:
   - Valores Nulos totales: 168
   - Precios como Texto (ej: '150 kr'): 652 filas
   - Fechas con formato incorrecto (ej: '/'): 0 filas


## 3: Auditoría de GASTOS (Finanzas)



In [4]:
finance_file = "../data/raw/finances/expenses_2024.csv"

if os.path.exists(finance_file):
    df_fin = pd.read_csv(finance_file)
    print("\n--- 💰 GASTOS OPERATIVOS (OPEX) ---")
    display(df_fin.head())
    
    # Verificar rangos de Renta
    rent_check = df_fin[df_fin['Category'] == 'Rent']
    print(f"\n📊 Estadísticas de Alquiler (Rent):")
    print(f"   - Mínimo: {rent_check['Amount'].min()} kr")
    print(f"   - Máximo: {rent_check['Amount'].max()} kr")
    print(f"   - Promedio: {round(rent_check['Amount'].mean(), 2)} kr")
else:
    print("❌ No encuentro el archivo de finanzas.")


--- 💰 GASTOS OPERATIVOS (OPEX) ---


,Expense_ID,Date,Store_ID,Category,Amount,Description
0,afd00dfb-984c-4eae-927c-d30be3b91d6b,2024-01-31,Stureplan_Flagship,Rent,73098.0,Monthly Rent invoice
1,46c767d2-9727-4d25-ab40-42104394d4b7,2024-01-31,Stureplan_Flagship,Electricity,27576.0,Monthly Electricity invoice
2,18ac83a8-3bdd-414b-9fb9-5b005dd432e0,2024-01-31,Stureplan_Flagship,Internet,1759.0,Monthly Internet invoice
3,a7d658c7-900a-4072-99eb-dc2dc8ebdde1,2024-01-31,Stureplan_Flagship,Software,15000.0,Monthly Software invoice
4,8cf1e13c-6d48-437f-8d7c-38f1e97fdd5e,2024-01-31,Stureplan_Flagship,Waste_Management,17448.0,Monthly Waste_Management invoice



📊 Estadísticas de Alquiler (Rent):
   - Mínimo: 40164.0 kr
   - Máximo: 78819.0 kr
   - Promedio: 58257.94 kr


## 4: Crear la "Zona de Aterrizaje" en la Nube

Si esta celda funciona, habrás modificado tu entorno en Google Cloud desde Python.

In [5]:
# Nombre del Dataset (Carpeta en BigQuery)
dataset_id = f"{PROJECT_ID}.raw_data_urban"

# Definir el objeto Dataset
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset.description = "Landing Zone: Datos crudos sin procesar"

try:
    # Intentar crearlo (si no existe)
    dataset = client.create_dataset(dataset, timeout=30)
    print(f"🎉 ¡ÉXITO! Dataset creado: {dataset.dataset_id}")
    print(f"   Ubicación: {dataset.location}")
except Exception as e:
    if "Already Exists" in str(e):
        print(f"✅ El dataset {dataset_id} ya existe (No es necesario crearlo de nuevo).")
    else:
        print(f"❌ Error creando dataset: {e}")

🎉 ¡ÉXITO! Dataset creado: raw_data_urban
   Ubicación: EU
